In [174]:
import os 
import pandas as pd 
import numpy as np 
import glob

In [175]:
out_list = glob.glob('/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter*/*filtered*')

In [176]:
out_list

['/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-C8-A12P-01A-11R-A115-07.all/G_filtered_df_TCGA-C8-A12P-01A-11R-A115-07_samp_chrt_norm_mot_unip_12f.tsv',
 '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-C8-A12P-01A-11R-A115-07.all/G_filtered_df_TCGA-C8-A12P-01A-11R-A115-07_samp_chrt_norm_mot_unip_13f.tsv',
 '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-C8-A12P-01A-11R-A115-07.all/G_filtered_df_TCGA-C8-A12P-01A-11R-A115-07_samp_chrt_norm_mot_unip.tsv',
 '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-C8-A12P-01A-11R-A115-07.all/G_filtered_df_TCGA-C8-A12P-01A-11R-A115-07_samp_chrt_norm_mot_uni

In [177]:
# Exclude backup files 
out_list = [filename for filename in out_list if 'orig' not in filename]
out_list = [filename for filename in out_list if '12f' not in filename]
out_list = [filename for filename in out_list if '13f' not in filename]

In [178]:
out_list

['/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-C8-A12P-01A-11R-A115-07.all/G_filtered_df_TCGA-C8-A12P-01A-11R-A115-07_samp_chrt_norm_mot_unip.tsv',
 '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-A2-A0SX-01A-12R-A084-07.all/G_filtered_df_TCGA-A2-A0SX-01A-12R-A084-07_samp_chrt_norm_mot_unip.tsv',
 '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-AO-A0JM-01A-21R-A056-07.all/G_filtered_df_TCGA-AO-A0JM-01A-21R-A056-07_samp_chrt_norm_mot_unip.tsv',
 '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-A2-A0D2-01A-21R-A034-07.all/G_filtered_df_TCGA-A2-A0D2-01A-21R-A034-07_samp_chrt_norm_mot_unip.tsv',


In [227]:
out_saved = []
for output_f in out_list[3:4]:
    print(output_f)

    with open(output_f, 'r') as fp: 
        file_13 = []
        file_14 = []
        file_12 = []

        for line in fp.readlines():
            line = line.split('\n')[0]
            if 'sample' in line:
                columns = line.split('\t')
            elif len(line.split('\t')) == 14:
                file_14.append(line.split('\t'))
            elif len(line.split('\t')) == 13:
                file_13.append(line.split('\t'))
            elif len(line.split('\t')) == 12:
                file_12.append(line.split('\t'))
    file_12 = pd.DataFrame(file_12)
    file_13 = pd.DataFrame(file_13)
    file_14 = pd.DataFrame(file_14)
    print(file_13.shape, file_14.shape, file_12.shape)

    # 1 Correct the missing columns 
    mycols_12 = [name for name in columns if name != 'Init_cancer']
    mycols_13 = columns 
    mycols_14 = columns + ['duplicate']

    if file_12.shape[0] > 0 : 
        file_12.columns = mycols_12
    if file_13.shape[0] > 0 : 
        file_13.columns = mycols_13
    if file_14.shape[0] > 0 : 
        file_14.columns = mycols_14

    # display(file_12.head())
    # display(file_13.head())
    # display(file_14.head())
    if file_13.shape[0] > 0 : 
        file_13['Init_cancer'] = file_13['Filter_Sample']
    if file_14.shape[0] > 0 : 
        file_14 = file_14.drop(['duplicate'], axis=1)
    df = pd.concat([file_13, file_12])
    print('not TCGA')
    print(df.shape)


    # 2. Suppress normals >0 for n >=1 samples
    df.loc[(df['reads_per_normal_sample'] == '0.0'), '#_normal_samples_allowed'] = '1'
    
    # 3. Remove duplicates 
    print('now unique')
    df = df.drop_duplicates(keep = 'last')
    print(df.shape)
    
    
    # 3. HACK, DO RERUNS!!!!!
    print('now hack')
    index_keep = df.loc[:, ['sample','mutation_mode','min_sample_reads','#_of_cohort_samples','reads_per_cohort_sample',
       '#_normal_samples_allowed','normal_cohort_id','reads_per_normal_sample']].drop_duplicates().index
    df = df.loc[index_keep]
    print(df.shape)

    # 4 join with batches
    print('with TCGA')
    df = pd.concat([file_14, df])
    print(df.shape)

    save_path = '.'.join(output_f.split('.')[:-1]) + '_cleaned20211129.tsv'
    out_saved.append(save_path)
    df.to_csv(save_path, index = None)

/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-A2-A0D2-01A-21R-A034-07.all/G_filtered_df_TCGA-A2-A0D2-01A-21R-A034-07_samp_chrt_norm_mot_unip.tsv
(326, 13) (0, 0) (0, 0)
not TCGA
(326, 13)
now unique
(207, 13)
now hack
(193, 13)
with TCGA
(193, 13)


In [226]:
out_saved

['/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-C8-A12P-01A-11R-A115-07.all/G_filtered_df_TCGA-C8-A12P-01A-11R-A115-07_samp_chrt_norm_mot_unip_cleaned20211129.tsv',
 '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-A2-A0SX-01A-12R-A084-07.all/G_filtered_df_TCGA-A2-A0SX-01A-12R-A084-07_samp_chrt_norm_mot_unip_cleaned20211129.tsv',
 '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-AO-A0JM-01A-21R-A056-07.all/G_filtered_df_TCGA-AO-A0JM-01A-21R-A056-07_samp_chrt_norm_mot_unip_cleaned20211129.tsv',
 '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-A2-A0D2-01A-21R-A034-07.all/G_filtered_df_TCGA-A2-A0D

In [228]:
df

,sample,mutation_mode,min_sample_reads,#_of_cohort_samples,reads_per_cohort_sample,#_normal_samples_allowed,normal_cohort_id,reads_per_normal_sample,Init_cancer,Filter_Sample,Filter_Sample_Cohort,Filter_Sample_Cohort_CohortNormal,Filter_Sample_Cohort_CohortNormal_Uniprot
0,TCGA-A2-A0D2-01A-21R-A034-07.all,ref,0.0,1,1.0,1,GTEXcore,0.0,1708726,1708726,1708511,135558,135384
1,TCGA-A2-A0D2-01A-21R-A034-07.all,ref,2.0,2,1.0,1,GTEXcore,0.0,1504678,1504678,1504288,107316,107193
2,TCGA-A2-A0D2-01A-21R-A034-07.all,ref,0.0,2,1.0,1,GTEXcore,0.0,1708726,1708726,1708057,135373,135199
4,TCGA-A2-A0D2-01A-21R-A034-07.all,ref,2.0,10,1.0,1,GTEXcore,0.0,1504678,1504678,1502178,106760,106637
5,TCGA-A2-A0D2-01A-21R-A034-07.all,ref,0.0,10,1.0,1,GTEXcore,0.0,1708726,1708726,1701716,133659,133489
...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,TCGA-A2-A0D2-01A-21R-A034-07.all,ref,0.0,10,5.0,2,matchedNormals,10.0,1708726,1708726,1632061,61298,61190
322,TCGA-A2-A0D2-01A-21R-A034-07.all,ref,0.0,10,5.0,1,matchedNormals,0.0,1708726,1708726,1632061,21550,21529
323,TCGA-A2-A0D2-01A-21R-A034-07.all,ref,0.0,10,5.0,2,matchedNormals,3.0,1708726,1708726,1632061,24484,24457
324,TCGA-A2-A0D2-01A-21R-A034-07.all,ref,0.0,10,5.0,10,matchedNormals,10.0,1708726,1708726,1632061,116248,116043


In [163]:
# # Read and reformat table 



# read_tables = {}

# for filename in out_list:
#     if '12f' in filename:
#         read_tables[filename] = pd.read_csv(filename, sep = '\t', header = None)
#     else:
#         read_tables[filename] = pd.read_csv(filename, sep = '\t')
#     if 'sample' in read_tables[filename].columns:
#         mycols = read_tables[filename].columns
        
# # 1. Correct tht missing column in 12f files 


# for filename in out_list:
#     if '12f' in filename:
#         read_tables[filename].columns = mycols_12
#         read_tables[filename]['Init_cancer'] = read_tables[filename]['Filter_Sample']
# # 2. Remove duplicates 
# for filename in out_list:
#     read_tables[filename] = read_tables[filename].drop_duplicates()

# # 3. Suppress normals >0 for n >=1 samples
